In [2]:
import pandas as pd
import boto3
import json

### 1- Load DWH Params from file

In [3]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

In [4]:
KEY            = config.get('AWS','KEY')
SECRET         = config.get('AWS','SECRET')


HOST           = config.get('CLUSTER','HOST')
DB_NAME        = config.get('CLUSTER','DB_NAME')
DB_USER        = config.get('CLUSTER','DB_USER')
DB_PASSWORD    = config.get('CLUSTER','DB_PASSWORD')
DB_PORT        = config.get('CLUSTER', 'DB_PORT')       
DWH_CLUSTER_TYPE = config.get('CLUSTER', 'DWH_CLUSTER_TYPE')
DWH_NUM_NODES =    config.get('CLUSTER','DWH_NUM_NODES')
DWH_NODE_TYPE =    config.get('CLUSTER', 'DWH_NODE_TYPE')
DWH_CLUSTER_IDENTIFIER = config.get('CLUSTER', 'DWH_CLUSTER_IDENTIFIER')
DWH_IAM_ROLE_NAME = config.get('CLUSTER', 'DWH_IAM_ROLE_NAME')
#DWH_ENDPOINT = config.get('CLUSTER', 'DWH_ENDPOINT')

#ARN
LOG_DATA       = config.get('S3','LOG_DATA') 
LOG_JSONPATH   = config.get('S3','LOG_JSONPATH')
SONG_DATA      = config.get('S3','SONG_DATA')

### 2 - Create clients for IAM, EC2, S3 and Redshift

In [5]:
ec2 = boto3.resource('ec2',
                    region_name = 'us-west-2',
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET
                    )

s3 = boto3.resource('s3',
                    region_name = 'us-west-2',
                   aws_access_key_id = KEY,
                   aws_secret_access_key= SECRET
                   )


iam = boto3.client('iam',
                  region_name = 'us-west-2',
                  aws_access_key_id = KEY,
                  aws_secret_access_key = SECRET
                  )

redshift = boto3.client('redshift',
                       region_name='us-west-2',
                       aws_access_key_id = KEY,
                       aws_secret_access_key = SECRET
                       )


### 3 -  IAM ROLE

- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [6]:
from botocore.exceptions import ClientError

try: 
    print("1.1 Creating a new IAM Role for the project")
    dwhProjectRole = iam.create_role(
    Path='/',
    RoleName = DWH_IAM_ROLE_NAME,
    Description = "Allows redshift clusters to call AWS services on your behalf",
    AssumeRolePolicyDocument = json.dumps(
    { 'Statement': [{'Action': 'sts:AssumeRole',
                    'Effect':'Allow',
                    'Principal': {'Service':'redshift.amazonaws.com'}}],
                     'Version': '2012-10-17'})
    )
    
except Exception as e:
    print(e)

1.1 Creating a new IAM Role for the project
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.


- Attach policy

In [7]:
try:
    print("1.2 Attaching policy")
    iam.attach_role_policy(RoleName = DWH_IAM_ROLE_NAME,
                          PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']

except Exception as e:
    print(e)

1.2 Attaching policy


- Get IAM role ARN

In [8]:
try:
    print("1.3 Get the IAM Role ARN")
    roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

except Exception as e:
    print(e)

1.3 Get the IAM Role ARN


In [9]:
print(roleArn)

arn:aws:iam::726510239907:role/dwhRole


### 4 - REDSHIFT CLUSTER

In [10]:
try:
    response = redshift.create_cluster(
        # HW
        ClusterType = DWH_CLUSTER_TYPE,
        NodeType = DWH_NODE_TYPE,
        NumberOfNodes = int(DWH_NUM_NODES),
        # Identifiers & Credentials
        DBName = DB_NAME,
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        MasterUsername = DB_USER,
        MasterUserPassword = DB_PASSWORD,
    
        # Roles (for S3 access)
        IamRoles = [roleArn]
    )
        
except Exception as e:
    print(e)
        
    

## 2.1 *Describe* the cluster to see its status


In [24]:
#- run this block several times until the cluster status becomes `Available`
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,redshift-cluster-project
5,Endpoint,"{'Address': 'dwhcluster.cmbss4e2jx12.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0256defe928b58140
7,NumberOfNodes,4


In [25]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", HOST)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.cmbss4e2jx12.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::726510239907:role/dwhRole


### 5 -  Open an incoming  TCP port to access the cluster ednpoint

In [26]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name ,  # TODO: fill out
        CidrIp='0.0.0.0/0',  # TODO: fill out
        IpProtocol='TCP',  # TODO: fill out
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0c2f9b6ea9bd30d70')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


**Delete cluster**

In [13]:
#-- Uncomment & run to delete the created resources
#redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'awsuser',
  'DBName': 'redshift-cluster-project',
  'Endpoint': {'Address': 'dwhcluster.cmbss4e2jx12.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 12, 10, 16, 1, 19, 791000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0c2f9b6ea9bd30d70',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-0256defe928b58140',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'thu:13:30-thu:14:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [